API 키 로드
`.env` 에 설정되어 있어야 합니다.
```
OPENAI_API_KEY=sk-proj-******** # Your Key
```

In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

LangSmith 추적을 설정합니다.(선택)

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("AI-SEARCH")

LangSmith 추적을 시작합니다.
[프로젝트명]
AI-SEARCH


In [3]:
# 라이브러리 선언
import sqlite3
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
import re

보통 Vector Store 를 생성한 임베딩과 동일한 임베딩을 사용합니다.

In [4]:
# 1. 임베딩(Embedding) 생성

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 1536 차원

# dimensions 파라미터 설정 (원하는 차원 수로 축소)
# embeddings = OpenAIEmbeddings(
#     model="text-embedding-3-small",
#     dimensions=512  # 512 차원으로 축소 (최대 1536까지 가능)
# )

사용자의 의도를 분석하여 필터를 생성합니다.

In [5]:
# 2. 의도 기반 필터링 함수 선언

def intent_based_filtering(query):
    """사용자 의도 분석을 통한 필터링"""

    filter_dict = {}

    # 가격필터
    filter_dict.update(intent_based_price_filtering(query))

    # 브랜드필터

    # 속성필터

    return filter_dict

def intent_based_price_filtering(query):
    """사용자 의도 분석을 통한 필터링"""

    filter_dict = {}

    # 가격 필터링을 위한 정규표현식 패턴
    # 1) N만원 이상 | N만원 부터
    price_pattern = r'(\d+)만원\s*(이상|부터)'
    price_match = re.search(price_pattern, query)
    
    if price_match:
        # SALE_PRC 키가 없으면 생성
        if "SALE_PRC" not in filter_dict:
            filter_dict["SALE_PRC"] = {}
        # 만원 단위를 원 단위로 변환 (예: 120만원 -> 1200000원)
        filter_dict["SALE_PRC"]["$gte"] = int(price_match.group(1)) * 10000
    
    # 2) N만원 이하 | N만원 까지
    price_pattern_lte = r'(\d+)만원\s*(이하|까지)'
    price_match_lte = re.search(price_pattern_lte, query)
    
    if price_match_lte:
        if "SALE_PRC" not in filter_dict:
            filter_dict["SALE_PRC"] = {}
        filter_dict["SALE_PRC"]["$lte"] = int(price_match_lte.group(1)) * 10000

    # 3) N만원 대
    price_pattern_range = r'(\d+)만원\s*대'
    price_match_range = re.search(price_pattern_range, query)
    
    if price_match_range:
        if "SALE_PRC" not in filter_dict:
            filter_dict["SALE_PRC"] = {}
        base_price = int(price_match_range.group(1)) * 10000
        filter_dict["SALE_PRC"]["$gte"] = base_price
        filter_dict["SALE_PRC"]["$lt"] = base_price * 2

    return filter_dict


### Vectorestore 검색 1 - 검색기만을 통한 검색
- 속도가 빠릅니다
- 검색어와 필터를 통해서만 검색결과를 조정할 수 있습니다.

In [ ]:
# DB 경로 설정
persist_directory = "../.db/faiss"

vectorstore = FAISS.load_local(
    persist_directory, 
    embeddings, 
    allow_dangerous_deserialization=True)

In [11]:

# 3. Vectorstore 검색 1 - 검색기만을 통한 검색

# 검색할 쿼리 설정 (원하는 검색어로 변경 가능)
query = "급배수 가능한 로봇청소기"  # 예시 쿼리: 원하는 검색어로 변경하세요

# 필터링 옵션 입력 받기
filter_dict = intent_based_filtering(query)
# filter_dict = {"GOODS_NO": '0031055945'}
# filter_dict = {
#     "SALE_PRC": {
#         "$gte": 1000000,  # 최소 가격
#         "$lte": 2000000   # 최대 가격
#     }
# }

print(f"필터: {filter_dict}")

# 쿼리와 유사한 상품 5개 검색
results_with_score = vectorstore.similarity_search_with_score(
    query=query, 
    filter = filter_dict, # 메타데이터 필터 적용
    k=5  # 반환할 결과 수
)

# 결과 출력
print(f"'{query}'와 유사한 상품 검색 결과 (최대 5개):")
print("-" * 60)

if results_with_score:
    for i, (doc, score) in enumerate(results_with_score, 1):
        print(f"[결과 {i}] 유사도: {score}") # 낮을수록 더 유사함
        print(f"{doc.page_content}")
        # print(f"메타데이터: {doc.metadata}")  # 모든 메타데이터 출력
        try:
            print(f"- 상품상태: {doc.metadata.get('GOODS_STAT_SCT_NM', '정보 없음')}")
            print(f"- 브랜드명: {doc.metadata.get('BRND_NM', '정보 없음')}")
            print(f"- 상품명: {doc.metadata.get('GOODS_NM', '정보 없음')}")
            print(f"- 품목정보: {doc.metadata.get('ARTC_INFO', '정보 없음')}")
            print(f"- 카테고리: {doc.metadata.get('CATEGORY_NMS', '정보 없음')}")
            print(f"- 판매가: {format(int(doc.metadata.get('SALE_PRC', 0)), ',')}원")
            print(f"- 할인가: {format(int(doc.metadata.get('DSCNT_SALE_PRC', 0)), ',')}원")
            print(f"- 최대혜택가: {format(int(doc.metadata.get('MAX_BENEFIT_PRICE', 0)), ',')}원")
            print(f"- 카드할인율: {doc.metadata.get('CARD_DC_RATE', '0')}%")
            print(f"- 할인카드: {doc.metadata.get('CARD_DC_NAME_LIST', '정보 없음')}")
            print(f"- 주요 특징 및 기능:")
            feature_values = doc.metadata['OPT_VAL_DESC'].split(',')
            feature_titles = doc.metadata['OPT_DISP_NM'].split(',')
            for i, (title, value) in enumerate(zip(feature_titles, feature_values)):
                print(f"  - {title} : {value}")
                if i == 3: break
            print(f"🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo={doc.metadata.get('GOODS_NO', '정보 없음')}")
        except Exception as e:
            print(f"메타데이터 처리 중 오류 발생: {str(e)}")
        print("-" * 60)
else:
    print("검색 결과가 없습니다.")
    print("다른 검색어로 다시 시도해보시거나, 검색어를 더 구체적으로 입력해주세요.")


필터: {}
'급배수 가능한 로봇청소기'와 유사한 상품 검색 결과 (최대 5개):
------------------------------------------------------------
[결과 1] 유사도: 0.8742688894271851
퓨어팩트 브랜드의 상품명 퓨어팩트 로봇청소기 흡입 물걸레 SLAM LDS센서 HEPA필터 자동충전 충돌추락방지 클린스테이션 NVPF01NVCHRC01NV. 모델명: ROBOT-CLEANER. 제품의 주요 특징은 품목: 로봇청소기, 청소방식: 흡입+걸레, 센서: LDS, 사용시간: 210분, 흡입력(W): 500W이상~, 소비전력: 900~1200W미만, 배터리 용량: 500mAh~9000mAh미만, 유무선: 무선, 무게: 3~6kg미만, 문턱넘기: ~1.5cm미만, 부가기능: 앱연동, 청소기능: 물걸레, 물분사가능, 흡입력 조절, 충전시간: 4시간 50분, 헤파필터 등급: H14, 필터 종류: H14급 헤파필터, 물통 용량: 300mll, 기능: 물걸레 . 카테고리: 청소기·생활가전. 가격 정보: 판매가 199,000원. 해시태그: #가성비#고성능필터#대용량#로봇청소기#어플리케이션연동#올인원클리닝시스템. 2023년 11월에 출시되었으며 최근 한 달 동안 1개 판매됨.
- 상품상태: 정상상품
- 브랜드명: 퓨어팩트
- 상품명: 퓨어팩트 로봇청소기 흡입 물걸레 SLAM LDS센서 HEPA필터 자동충전 충돌추락방지 클린스테이션 NVPF01NVCHRC01NV
- 품목정보: 정보 없음
- 카테고리: 정보 없음
- 판매가: 206,024원
- 할인가: 199,000원
- 최대혜택가: 199,000원
- 카드할인율: 0%
- 할인카드: 
- 주요 특징 및 기능:
  - 품목 : 로봇청소기
  - 청소방식 : 흡입+걸레
  - 센서 : LDS
  - 사용시간 : 210분
🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=0021221690
------------

### Vectorestore 검색 2 - LLM 과 검색기를 체인으로 생성하여 검색 
- 속도가 느립니다.
- 검색어와 필터를 뿐 만아니라 프롬프트로도 검색결과를 조정할 수 있습니다.
- 친절하고 감성적인 답변이 가능합니다.

In [7]:
# 1. 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are a product search assistant for an electronics e-commerce platform. 
Use the following pieces of retrieved context to answer the question. 
Search up to 10 products.
Kindly explain why you've been searching for the product.
And be sure to include the following meta information:
상품명
- 상품상태
- 브랜드명
- 판매가
- 할인가
- 최대혜택가
- 할인카드
- 주요 특징 및 기능
  - 
  -
  -
🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=상품번호
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)


In [8]:
# 2. LLM 과 체인생성

# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 의도 기반 필터링 함수 선언
def get_retriever_with_filter(query):
    filter_dict = intent_based_filtering(query)

    print(f"필터정보:{filter_dict}")

    return vectorstore.as_retriever(
        search_kwargs={
            "k": 5,
            "filter": filter_dict
        }
    )

# 체인(Chain) 생성
chain = (
    {"context": lambda x: get_retriever_with_filter(x).get_relevant_documents(x), 
     "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [9]:
# 3. 체인 실행
query = "200만원 이하 삼성 스마트 세탁기"  # 예시 쿼리: 원하는 검색어로 변경하세요

response = chain.invoke(query)
print(response)

필터정보:{'SALE_PRC': {'$lte': 2000000}}


C:\Users\twili\AppData\Local\Temp\ipykernel_16000\2393032965.py:21: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  {"context": lambda x: get_retriever_with_filter(x).get_relevant_documents(x),


다음은 200만원 이하의 삼성 스마트 세탁기 목록입니다. 이 제품들은 다양한 세탁 용량과 스마트 기능을 제공하여 사용자의 편리함을 높입니다.

1. **상품명**: 그랑데 일반세탁기 WA18CG6K46BV 18kg
   - **상품상태**: 정상상품
   - **브랜드명**: 삼성전자
   - **판매가**: 741,800원
   - **할인가**: 667,620원
   - **최대혜택가**: 634,240원
   - **할인카드**: 롯데카드
   - **주요 특징 및 기능**:
     - 세탁 용량: 17~21kg
     - 스마트기능: 코스다운로드, 스마트컨트롤, 스마트체크
     - 모터 종류: 인버터 DD모터
   - 🔗 [상품보러가기](https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=0026697878)

2. **상품명**: 그랑데 일반세탁기 WA18CG6741BW 18kg
   - **상품상태**: 정상상품
   - **브랜드명**: 삼성전자
   - **판매가**: 1,218,500원
   - **할인가**: 1,096,650원
   - **최대혜택가**: 1,041,820원
   - **할인카드**: 롯데카드
   - **주요 특징 및 기능**:
     - 세탁 용량: 18kg
     - 스마트기능: 스마트체크, 스마트타이머
     - 모터 종류: 인버터 DD모터
   - 🔗 [상품보러가기](https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=0026697880)

3. **상품명**: 삼성 그랑데 통버블 세탁기 18kg WA18CG6K46BV
   - **상품상태**: 정상상품
   - **브랜드명**: 삼성전자
   - **판매가**: 822,000원
   - **할인가**: 748,020원
   - **최대혜택가**: 710,620원
   - **할인카드**: 롯데카드
   - **주요 특징 및 기능**:
     - 세탁 